<div style="text-align: left; background-color:#FFFFCC; font-family:Arial; color:#526085; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 58px;border-style: solid;border-color: dark green;">Lab 02: Working with time-series data</div>
<div style="text-align: left; background-color:#99FFFF; font-family: Arial; color:#551184; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 18px">Bộ môn: Trực quan hóa dữ liệu | Nhóm 8</div>

# <div style="text-align: left; background-color:#0C6A86; font-family:Arial; color:white; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 30px;border-style: solid;border-color: black;">PHẦN 2. Trực quan hóa dữ liệu</div>

# Import các thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
import plotly.offline as pyo

# Nội dung chính

## Đọc dữ liệu

In [3]:
df = pd.read_csv("Data/data_18_19_processed.csv")
df.head()

,Reference Number,Grid Ref: Easting,Grid Ref: Northing,Number of Vehicles,1st Road Class,1st Road Class & No,Road Surface,Lighting Conditions,Weather Conditions,Local Authority,Type of Vehicle,Casualty Class,Casualty Severity,Sex of Casualty,Age of Casualty,Full Time
0,51B0230,433936,428874,1,6,U,1,4,1,E08000035,9,3,3,2,87,2018-01-11 07:00:00
1,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,60,2018-01-11 08:55:00
2,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,51,2018-01-11 08:55:00
3,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,50,2018-01-11 08:55:00
4,51B0349,430076,433872,1,6,U,2,1,1,E08000035,11,2,3,2,51,2018-01-11 08:55:00


## Trực quan hóa dữ liệu

### Biểu đồ 

**Tiêu đề:** Thống kê số tai nạn tại các khung giờ của các ngày trong tuần.

**Loại biểu đồ:** Heatmap.

**Lý do lựa chọn:**
- **Trực quan hóa dễ hiểu và chi tiết**: Heatmap là một công cụ trực quan hóa dữ liệu mạnh mẽ, giúp biểu diễn thông tin theo cách dễ nhìn và dễ hiểu. Biểu đồ này sử dụng các màu sắc khác nhau để thể hiện mức độ thay đổi của số lượng tai nạn qua các khung giờ và các ngày trong tuần, giúp người đọc dễ dàng nhận thấy những thời điểm có nguy cơ cao.

- **Dễ thấy sự khác biệt và xu hướng**: Sự thay đổi màu sắc trên Heatmap giúp người đọc dễ dàng nhận ra các xu hướng và điểm nóng trong dữ liệu. Chẳng hạn, những ô màu đậm hơn chỉ ra thời điểm và ngày có số lượng tai nạn cao hơn, cho phép dễ dàng xác định các khung giờ và ngày nguy hiểm nhất.

- **Phân tích theo thời gian và không gian**: Với Heatmap, chúng ta có thể phân tích dữ liệu tai nạn theo cả thời gian (giờ trong ngày) và không gian (ngày trong tuần). Điều này rất hữu ích để phát hiện ra các mẫu hoặc xu hướng tiềm ẩn trong dữ liệu, từ đó có thể đề xuất các biện pháp cải thiện an toàn giao thông.

**Trực quan hóa:**

In [16]:
# Create an empty 7x24 matrix
matrix_ = np.zeros((7, 24), dtype=int)

# Fill in the matrix
df['Full Time'] = pd.to_datetime(df['Full Time'])
full_time = df['Full Time']

for time in full_time:
    i = time.weekday()
    j = time.hour
    matrix_[i][j] += 1

# Convert the matrix to a DataFrame for easier manipulation
day_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
matrix_df = pd.DataFrame(matrix_, columns=range(24))
matrix_df['Day'] = day_of_week

# Melt the DataFrame to have a long format
matrix_long = matrix_df.melt(id_vars='Day', var_name='Hour', value_name='Count')

# Create the heatmap using Plotly
fig = px.imshow(
    matrix_,
    labels=dict(x="Thời gian (giờ)", y="Ngày trong tuần", color="Số lượng tai nạn"),
    x=list(range(24)),
    y=day_of_week,
    color_continuous_scale='reds',
    aspect="auto"
)

# Update layout for better visualization
fig.update_layout(
    title="Thống kê tai nạn tại các khung giờ của các ngày trong tuần",
    xaxis_title="Thời gian (giờ)",
    yaxis_title="Ngày trong tuần"
)

# Adding hover data to see exact counts
fig.update_traces(
    hovertemplate="Thời gian: %{x}h<br>Ngày: %{y}<br>Số lượng tai nạn: %{z}<extra></extra>"
)

# Show the plot
fig.show()


**Kỹ thuật trực quan sử dụng:**

- ***Màu sắc***: Heatmap sử dụng các sắc độ của màu đỏ để biểu thị số lượng tai nạn giao thông tại các khung giờ và các ngày trong tuần. Các sắc độ màu từ nhạt đến đậm thể hiện mức độ gia tăng của số lượng tai nạn:
    - Màu đỏ nhạt: số lượng tai nạn ít.
    - Màu đỏ đậm: số lượng tai nạn nhiều.
    - Sắc độ màu càng đậm, số lượng tai nạn càng cao; sắc độ màu càng nhạt, số lượng tai nạn càng thấp.

- ***Kỹ thuật Manipulate View:***
  + **Hiển thị chi tiết qua tooltip**: Khi người dùng trỏ chuột vào từng ô trong heatmap, một tooltip sẽ xuất hiện hiển thị số lượng tai nạn cụ thể tại khung giờ và ngày trong tuần đó. Điều này giúp người dùng có thể xem thông tin chi tiết mà không cần phải xem toàn bộ dữ liệu cùng một lúc.
  + **Tùy chỉnh giao diện hiển thị**: Heatmap có thể được thiết kế để cho phép người dùng lọc hoặc thay đổi phạm vi thời gian hoặc ngày trong tuần, từ đó tập trung vào các khoảng thời gian hoặc ngày cụ thể mà họ quan tâm. Ví dụ, người dùng có thể chọn xem chỉ số tai nạn trong các ngày cuối tuần hoặc trong giờ cao điểm.

$\Rightarrow$ Giúp đồ thị heatmap trở nên rõ ràng, trực quan, và tương tác thân thiện với người dùng, cho phép họ dễ dàng nắm bắt được các xu hướng và điểm nóng về số lượng tai nạn giao thông.

**Nhận xét:**

- Nhìn chung, số lượng tai nạn giao thông có sự phân bố không đồng đều theo các khung giờ trong ngày và các ngày trong tuần.
- Số lượng tai nạn cao nhất thường xảy ra vào giờ cao điểm buổi sáng và buổi chiều (khoảng 7-9 giờ sáng và 15-17 giờ chiều). Đây là các khoảng thời gian mà mật độ giao thông cao nhất do mọi người di chuyển đến và rời khỏi nơi làm việc.
- Thứ Ba và thứ Sáu có số lượng tai nạn cao nhất trong tuần. Điều này có thể do thứ Ba là ngày làm việc đầu tuần sau thứ Hai (thường có ít tai nạn hơn) và thứ Sáu là ngày cuối tuần, khi mọi người có xu hướng vội vàng hoàn thành công việc để về nhà hoặc chuẩn bị cho các hoạt động cuối tuần.
- Thứ Hai và Chủ Nhật có số lượng tai nạn thấp hơn so với các ngày khác trong tuần. Thứ Hai có thể ít tai nạn hơn vì mọi người thường cẩn thận hơn sau kỳ nghỉ cuối tuần. Chủ Nhật, là ngày nghỉ, mật độ giao thông thấp hơn nên số lượng tai nạn cũng giảm.

$\Rightarrow$ Từ đó cho thấy tầm quan trọng của việc quản lý và kiểm soát giao thông vào các khung giờ cao điểm và các ngày có mật độ giao thông cao để giảm thiểu số lượng tai nạn. Cũng cần có các biện pháp phòng ngừa tai nạn đặc biệt vào thứ Ba và thứ Sáu.